# Motif Mark Test Notebook

## Building the Script

In [1]:
import re
import cairo
import math

In [104]:
#!/usr/bin/env python

# motif_mark.py

import re
import sys
import argparse
import cairo
import math
import random

#parser = argparse.ArgumentParser(description="Creates a visualization of one or more motifs across a sequence or multiple sequences provided in UCSC format. Requires sequences and a list of motifs to be visualized.")
#parser.add_argument('-f','--file', help='absolute path to FASTA file of sequences to be searched.', required=True, type=str)
#parser.add_argument('-o','--out_dir', help='absolute path to the output directory.', required=True, type=str)
#parser.add_argument('-m','--motifs', help='absolute path to the text file containing the list of motifs to be visualized, formatted as one motif per line.', required=True, type=str)
#args = parser.parse_args()

#####################################################################################
##### Define Higher Order Functions #################################################
#####################################################################################

def fasta_twofer(filepath):
    '''Takes in a FASTA file and creates a new FASTA file called "sequences_twofer.fasta"
    that only has two lines for every FASTA entry.'''
    #with open(filepath,"r+") as fasta, open(args.out_dir/"sequences_twofer.fasta", "w+") as out:
    first_line = True
    with open(filepath,"r") as fasta, open("./sequences_twofer.fasta", "w+") as out:
        for line in fasta:
            if first_line and line.startswith(">"):     ### If the first line, print line without "\n" because print statement appends "\n"
                out.write(line.strip())
                out.write("\n")
                first_line = False     ### Never run through this part of the looping system again
            elif not first_line and line.startswith(">"):     ### Print "\n" preceeding all deflines
                out.write("\n")
                out.write(line.strip())
                out.write("\n")
            else:
                out.write(line.strip("\n"))     ### Strip "\n" from all sequence lines

def count_lines(infile):
    '''Opens the input file and returns the number of lines in the file.'''
    with open(infile) as file:
        for i, line in enumerate(file):
            pass
    return i + 1

def motif2regex(file):
    '''Takes the list of motifs and returns a dictionary of regular expressions that will search for that motif in
    a given sequence, regardless of case. Keys are motifs; values are regex motifs.'''
    motif_dict = {}
    with open(file) as motif_list:
        for motif in motif_list:
            motif = motif.strip("\n")
            motif_upper = motif.upper()
            motif_components = list(motif_upper)
            for i in range(0, len(motif_components)):
                motif_components[i] = iupac.get(motif_components[i])
                regex_motif = "".join(motif_components)
                motif_dict[motif] = regex_motif
    return motif_dict

def lengths_intex(sequence):
    '''From a given UCSC sequence, i.e. lower case intron, upper case exon, lower case intron,
    this function returns a list of length four: length of the sequence, length of the first intron,
    length of the exon, and length of the second intron.'''
    sequence = sequence.strip()
    pattern = re.compile(r'([a-z]+)([A-Z]+)([a-z]+)')
    res = re.match(pattern, sequence)
    lengths = [sequence, len(sequence), len(res.group(1)), len(res.group(2)), len(res.group(3))]
    return lengths

def sequence_parser(file):
    with open(file, "r+") as file:
        seq_details = {}
        header = ""
        sequence = ""
        first_line = True
        for line in file:
            if line.startswith(">") == True:
                line = line.strip()
                if first_line == False:
                    seq_details[header] = lengths_intex(sequence)
                header = line
                sequence = ""
            else:
                sequence = line
                first_line = False
        seq_details[header] = lengths_intex(sequence)
    return seq_details

def motif_position(motif_dict, header, sequence):
    '''Takes a motif in regular-expression format and searches for all instances of that motif
    in the provided sequence. Returns a list of 2-item lists containing start positions and lengths.
    Since Python idexes starting at 0, 1 is added to the start position to account for pixels when making the image.'''
    positions = {}
    for motif in motif_dict.keys():   
        for match in re.finditer(motif_dict[motif], sequence):
            s = match.start()
            e = match.end()
            if header + "_" + motif in positions:
                positions[header + "_" + motif].append([s + 1, e - s])
            elif header + "_" + motif not in positions:
                positions[header + "_" + motif] = [[s + 1, e - s]]
    return positions
        
def get_all_positions(fasta_f, motif_f):
    seq_details = sequence_parser(fasta_f)
    motif_dict = motif2regex(motif_f)
    all_positions = []
    for info in seq_details.items():
        # info[0] is the header
        # info[1] is the list of details
        # info[1][0] is the sequence
        # info[1][1] is the sequence length
        # info[1][2] is the length of intron 1
        # info[1][3] is the length of the exon
        # info[1][4] is the length of intron 2
        positions = motif_position(motif_dict, header = info[0], sequence = info[1][0])
        positions["intron1"] = info[1][2]
        positions["exon"] = info[1][3]
        positions["intron2"] = info[1][4]
        all_positions.append(positions)
    return all_positions

#####################################################################################
##### Check File Sizes ##############################################################
#####################################################################################

#if count_lines(args.file) > 20:
#    raise ValueError("ERROR: Exiting program. Too many sequences were supplied (maximum 10)")
#if count_lines(args.motifs) > 10:
#    raise ValueError("ERROR: Exiting program. Too many motifs were supplied (maximum 10)")

#####################################################################################
##### Make IUPAC Dictionary #########################################################
#####################################################################################

### IUPAC ambiguity codes and associated regular expressions for ignoring case
iupac = {"A":"[Aa]", "T":"[TtUu]", "C":"[Cc]", "G":"[Gg]", "U":"[TtUu]",
         "M":"[AaCc]", "R":"[AaGg]", "W":"[AaTtUu]",
         "S":"[CcGg]", "Y":"[CcTtUu]", "K":"[GgTtUu]",
         "V":"[AaCcGg]", "H":"[AaCcTtUu]", "D":"[AaGgTtUu]", "B":"[CcGgTtUu]",
         "N":"[AaTtCcGgUu]"}

####################################################################################
##### Find Motifs in Sequences ######################################################
#####################################################################################



#####################################################################################
##### Generate Visualizations #######################################################
#####################################################################################



In [105]:
fasta_twofer("./sequence.fasta")

all_positions = get_all_positions(fasta_f = "./sequences_twofer.fasta", motif_f = "./motifs.txt")
all_positions

[{'>INSR chr19:7149896-7151209 (reverse complement)_GCAUG': [[453, 5],
   [535, 5]],
  '>INSR chr19:7149896-7151209 (reverse complement)_UGCAUG': [[452, 6]],
  '>INSR chr19:7149896-7151209 (reverse complement)_ygcy': [[8, 4],
   [131, 4],
   [148, 4],
   [218, 4],
   [339, 4],
   [380, 4],
   [384, 4],
   [407, 4],
   [418, 4],
   [471, 4],
   [485, 4],
   [509, 4],
   [572, 4],
   [596, 4],
   [618, 4],
   [642, 4],
   [726, 4],
   [826, 4]],
  'exon': 36,
  'intron1': 317,
  'intron2': 505},
 {'>MBNL chr3:152432504-152433226 (reverse complement)_ygcy': [[18, 4],
   [415, 4],
   [466, 4]],
  'exon': 204,
  'intron1': 306,
  'intron2': 213},
 {'>ADD3 chr10:111891895-111892326_GCAUG': [[35, 5], [137, 5]],
  '>ADD3 chr10:111891895-111892326_UGCAUG': [[136, 6]],
  '>ADD3 chr10:111891895-111892326_ygcy': [[38, 4],
   [50, 4],
   [118, 4],
   [170, 4],
   [197, 4],
   [429, 4]],
  'exon': 96,
  'intron1': 168,
  'intron2': 168}]

In [82]:
seq_details

{'ADD3 chr10:111891895-111892326': ['aatgtataattatggatatatgggataactgttagcatgctcagctcactgctgaagaatttatcatctctttgtatacaggcatttgatgtatgcactaacctccctaaaatcatatgctgctttgttttgttttgcatggcttttaactaaactcttatccaacagATGCTGAGCAGGAATTACTCTCAGATGACGCTTCATCTGTTTCACAAATTCAGTCTCAAACTCAGTCACCGCAAAATGTCCCTGAAAAATTAGAAGgtactcaatgtaatttcccacatagcattcactgagttagtcttgagtctgtccctctgtgttttgttttcacgtgaggaagttgaatacctcatcacagtaagttttccatattttacttatatctcccaataattacatattttatatcattaaaaatggggcgct',
  432,
  168,
  96,
  168],
 'INSR chr19:7149896-7151209 (reverse complement)': ['aaaattctgccagacttggagaagtggctgagtcagttgtgatgtccacatgtagtcacgtttgacatcccagggccacctcagcaggccgtctctggggagaattttctctgatttcttccccttcccttgctggacccctgcacctgctggggaagatgtagctcactccgtctagcaagtgatgggagcgagtggtccagggtcaaagccagggtgcccttactcggacacatgtggcctccaagtgtcagagcccagtggtctgtctaatgaagttccctctgtcctcaaaggcgttggttttgtttccacagAAAAACCTCTTCAGGCACTGGTGCCGAGGACCCTAGgtatgactcacctgtgcgacccctggtgcctgctccgcgcagggccggcggcgtgccaggcagatgcctcggagaacccaggggtttctgtgg

# `Pycairo` Testing

In [138]:
def draw_intex(intron1, exon, intron2, start):
    context.set_source_rgb(0, 0, 0)
    context.set_line_width(1)
    context.move_to(start[0], start[1])
    context.line_to(start[0] + intron1 + exon + intron2, start[1])
    context.stroke()
    
    context.set_line_width(10)
    context.move_to(start[0] + intron1, start[1])
    context.line_to(start[0] + intron1 + exon, start[1])
    context.stroke()


def draw_motifs(pos_list, context, start, motif):
    '''draws the position of the motifs on the introns/exon. Returns the color values for motif'''
    r = random.random()
    g = random.random()
    b = random.random()

    context.set_line_width(12)
    context.set_source_rgb(r, g, b)


    for pos in pos_list:
        context.move_to(start[0] + pos, start[1])
        context.line_to(start[0] + pos + len(motif), start[1])
        context.stroke()

        context.move_to(start[0] + pos, start[1] - 20)
        context.show_text(str(pos))

    return [red, green, blue]
    
surface = cairo.SVGSurface("test.svg", 1600, 1000)
context = cairo.Context(surface)
context.set_line_width(1)
start = [300,100]

for one_position in all_positions:
    draw_intex(one_position["intron1"], one_position["exon"], one_position["intron2"], start)
    for motif_group in one_position.items():
        if motif_group[0].startswith(">"):
            motif = motif_group[0].split("_")[1]
            details = motif_group[0].split("_")[0][1:]
            gene = details.split(" ")[0]
            info = " ".join(details.split(" ")[1:])
            
            context.set_source_rgb(0, 0, 0)
            context.move_to(start[0] - 275, start[1] - 5)
            context.show_text(gene)
            context.move_to(start[0] - 275, start[1] + 5)
            context.show_text(info)
            
            r = random.random()
            g = random.random()
            b = random.random()
            context.set_line_width(12)
            context.set_source_rgb(r, g, b)    

            for m in motif_group[1]:
                context.move_to(start[0] + m[0], start[1])
                context.line_to(start[0] + m[0] + m[1], start[1])
                context.stroke()
                
                context.move_to(start[0] + m[0], start[1] - 20)
                context.show_text(str(m[0]))
    start = [start[0], start[1] + 150]

surface.finish()

In [133]:
for one_position in all_positions:
    for motif_group in one_position.items():
        if motif_group[0].startswith(">"):
            motif = motif_group[0].split("_")[1]
            details = motif_group[0].split("_")[0][1:]
            gene = details.split(" ")[0]
            info = " ".join(details.split(" ")[1:])
            print(gene)
            print(info)
            #for m in motif_group[1]:
             #   print(m)

INSR
chr19:7149896-7151209 (reverse complement)
INSR
chr19:7149896-7151209 (reverse complement)
INSR
chr19:7149896-7151209 (reverse complement)
MBNL
chr3:152432504-152433226 (reverse complement)
ADD3
chr10:111891895-111892326
ADD3
chr10:111891895-111892326
ADD3
chr10:111891895-111892326


## In-Line Testing

In [2]:
### IUPAC ambiguity codes and associated regular expressions for ignoring case
iupac = {"A":"[Aa]", "T":"[TtUu]", "C":"[Cc]", "G":"[Gg]", "U":"[TtUu]",
         "M":"[AaCc]", "R":"[AaGg]", "W":"[AaTtUu]",
         "S":"[CcGg]", "Y":"[CcTtUu]", "K":"[GgTtUu]",
         "V":"[AaCcGg]", "H":"[AaCcTtUu]", "D":"[AaGgTtUu]", "B":"[CcGgTtUu]",
         "N":"[AaTtCcGgUu]"}

## `Pycairo` Example from Leslie

In [12]:
width, height = 800, 500

#create the coordinates to display your graphic, desginate output
surface = cairo.SVGSurface("example.svg",width, height)
#create the coordinates you will be drawing on (like a transparency) - you can create a transformation matrix
context = cairo.Context(surface)
#context.scale(width,height) #will set your drawing surface to a 0.0-1.0 scale

#Need to tell cairo where to put the brush, the color and width, and the shape you want it to draw
#draw a line
context.set_line_width(1)
context.move_to(50,25)        #(x,y)
context.line_to(450,25)
context.stroke()

#draw a rectangle
context.rectangle(100,100,150,350)        #(x0,y0,x1,y1)
context.fill()

context.move_to(25,250)
context.set_source_rgb(.25,.5,.5)
context.set_line_width(5)
context.curve_to(100,400,400,100,750,250)
context.stroke()

context.set_source_rgb(0,.6,.8)
context.arc(500, 300, 50, 0, 2*math.pi)
context.fill_preserve()            #draws filled circle, but preserves for later drawing
context.set_source_rgb(.3,.3,.3)
context.stroke()

#write out drawing
surface.finish()

In [11]:
WIDTH, HEIGHT = 256, 256

surface = cairo.ImageSurface (cairo.FORMAT_ARGB32, WIDTH, HEIGHT)
ctx = cairo.Context (surface)

ctx.scale (WIDTH, HEIGHT) # Normalizing the canvas

pat = cairo.LinearGradient (0.0, 0.0, 0.0, 1.0)
pat.add_color_stop_rgba (1, 0.7, 0, 0, 0.5) # First stop, 50% opacity
pat.add_color_stop_rgba (0, 0.9, 0.7, 0.2, 1) # Last stop, 100% opacity

ctx.rectangle (0, 0, 1, 1) # Rectangle(x0, y0, x1, y1)
ctx.set_source (pat)
ctx.fill ()

ctx.translate (0.1, 0.1) # Changing the current transformation matrix

ctx.move_to (0, 0)
# Arc(cx, cy, radius, start_angle, stop_angle)
ctx.arc (0.2, 0.1, 0.1, -math.pi/2, 0)
ctx.line_to (0.5, 0.1) # Line to (x,y)
# Curve(x1, y1, x2, y2, x3, y3)
ctx.curve_to (0.5, 0.2, 0.5, 0.4, 0.2, 0.8)
ctx.close_path ()

ctx.set_source_rgb (0.3, 0.2, 0.5) # Solid color
ctx.set_line_width (0.02)
ctx.stroke ()

surface.write_to_png ("example2.png") # Output to PNG